In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from transformers import RobertaTokenizer, RobertaModel
import pandas as pd
from PIL import Image


In [19]:
! pip install -q gdown
! pip install -q torchmetrics

!wget --no-check-certificate -O bully_data.zip "https://iitgoffice-my.sharepoint.com/:u:/g/personal/sjana_iitg_ac_in/EWdMrS9zHgBHnz4TTckw14kB14O8j0IbR_D-fBowyw7T7A?e=8H4Wpw&download=1"

!gdown "https://drive.google.com/uc?id=1RpvrH0OJXBnJDzAFerqBb7v1ZKLbxTG8"
!gdown "https://drive.google.com/uc?id=19oLWMCuVHye63YvKBrG7-EbW4OU_VQOQ"

!rm -rf bully_data/
!mkdir bully_data
!mkdir bully_data/data
#!mkdir bully_data/test
!unzip bully_data.zip -d bully_data/data/ | tqdm >/dev/null
!unzip Cyberbully_corrected_emotion_sentiment_v2.zip -d bully_data/ | tqdm >/dev/null

--2024-11-24 10:18:02--  https://iitgoffice-my.sharepoint.com/:u:/g/personal/sjana_iitg_ac_in/EWdMrS9zHgBHnz4TTckw14kB14O8j0IbR_D-fBowyw7T7A?e=8H4Wpw&download=1
Resolving iitgoffice-my.sharepoint.com (iitgoffice-my.sharepoint.com)... 13.107.138.10, 13.107.136.10, 2620:1ec:8fa::10, ...
Connecting to iitgoffice-my.sharepoint.com (iitgoffice-my.sharepoint.com)|13.107.138.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/sjana_iitg_ac_in/Documents/bully_data-20240404T103912Z-001.zip?ga=1 [following]
--2024-11-24 10:18:03--  https://iitgoffice-my.sharepoint.com/personal/sjana_iitg_ac_in/Documents/bully_data-20240404T103912Z-001.zip?ga=1
Reusing existing connection to iitgoffice-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 124138703 (118M) [application/x-zip-compressed]
Saving to: 'bully_data.zip'

bully_data.zip      100%[===================>] 118.39M  28.7MB/s    in 4.3s    

2024-11-24 10:18:08 (27.5 MB/s) - 'bully

In [21]:
import pandas as pd
import pickle

excel_path = "/kaggle/working/bully_data/data/bully_data/Copy of Cyberbully_corrected_emotion_sentiment.xlsx"
image_path = "/kaggle/working/bully_data/data/bully_data/"

# Load the Excel file
df = pd.read_excel(excel_path)
df = df.drop(columns=['Unnamed: 10', 'Unnamed: 11'])
# Display the first few rows to ensure it is loaded correctly
# print(df)

df_cleaned = df.dropna()
df=df_cleaned
# print(df)
import os
import pandas as pd

# Assuming your DataFrame is called df and contains the 'img_id' column
# Assuming image paths are in a directory (img_dir) and filenames correspond to 'img_id'

img_dir = image_path
# Define a function to check if the image size is zero
def is_zero_size(img_id, img_dir):
    img_path = os.path.join(img_dir, img_id)
    return os.path.exists(img_path) and os.path.getsize(img_path) == 0

# Filter out rows with zero-size images
df['is_zero_size'] = df['Img_Name'].apply(lambda img_id: is_zero_size(img_id, img_dir))
df_filtered = df[df['is_zero_size'] == False].drop(columns='is_zero_size')

# Now, df_filtered contains only rows with non-zero-size images
# print(df_filtered)
df=df_filtered
df_cleaned = df[df['Img_Name'] != '2644.jpg']
# df_cleaned = df[df['Img_Name'] != '3805.jpg']
df=df_cleaned


In [22]:
df.shape

(3078, 10)

In [23]:
emotion_mapping = {
    "Disgust": "Negative",
    "Ridicule": "Negative",
    "Sadness": "Negative",
    "Surprise": "Neutral",
    "Anticipation": "Positive",
    "Angry": "Negative",
    "Happiness": "Positive",
    "Other": "Neutral",
    "Trust": "Positive",
    "Fear": "Negative"
}

In [24]:
df['Emotion'] = df['Emotion'].map(emotion_mapping)


/tmp/ipykernel_30/4169271968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Emotion'] = df['Emotion'].map(emotion_mapping)


In [25]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [26]:
df.shape

(3078, 10)

In [27]:
df.head()

,Img_Name,Img_Text,Img_Text_Label,Img_Label,Text_Label,Sentiment,Emotion,Sarcasm,Harmful_Score,Target
0,0.jpg,Shivam @shivamishraa Girls be named naina and ...,Bully,Nonbully,Bully,Negative,Negative,Yes,Partially-Harmful,Individual
2,2.jpg,For Boyfriend For Bestfriend DESI ADUKT TROLLS,Bully,Bully,Nonbully,Negative,Negative,No,Partially-Harmful,Society
4,4.jpg,not_shubham14 @mentally_dank Kids at Marine Dr...,Bully,Nonbully,Nonbully,Negative,Negative,No,Partially-Harmful,Individual
5,5.jpg,what if we use 100% of our brain?,Bully,Bully,Nonbully,Negative,Neutral,No,Partially-Harmful,Individual
6,6.jpg,If the opposite of Con is Pro Is Congress the ...,Bully,Nonbully,Bully,Negative,Negative,No,Partially-Harmful,Organization


In [41]:
class MemeDataset(Dataset):
    def __init__(self, dataframe, transform):
        self.dataframe = dataframe
        self.transform = transform
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        self.img_folder = image_path
        # Define label mappings
        self.text_label_mapping = {
            "Bully": 1,
            "Nonbully": 0
        }
        
        self.sentiment_mapping = {
            "Positive":1,
            "Neutral": 0,
            "Negative": 2
        }
        
        # self.emotion_mapping = {
        #     "Disgust": 0,
        #     "Ridicule": 1,
        #     "Sadness": 2,
        #     "Surprise": 3,
        #     "Anticipation": 4,
        #     "Angry": 5,
        #     "Happiness": 6,
        #     "Other": 7,
        #     "Trust": 8,
        #     "Fear": 9
        # }
        
        self.emotion_mapping = {
            "Positive":1,
            "Neutral": 0,
            "Negative": 2
        }
        self.sarcasm_mapping = {
            "Yes": 1,
            "No": 0
        }
        
        self.harmful_score_mapping = {
            "Harmless": 0,
            "Partially-Harmful": 1,
            "Very-Harmful": 2
        }
        
        self.target_mapping = {
            "Individual": 0,
            "Society": 1,
            "Organization": 2,
            "Community": 3
        }
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        # Load image
        img_name = self.dataframe.iloc[idx]['Img_Name']
        img_path = os.path.join(self.img_folder, img_name)
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        
        # Load and tokenize text
        text = self.dataframe.iloc[idx]['Img_Text']
        inputs = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
        
        # Get labels and apply mappings
        sentiment_label = torch.tensor(self.sentiment_mapping[self.dataframe.iloc[idx]['Sentiment']], dtype=torch.long)
        emotion_label = torch.tensor(self.emotion_mapping[self.dataframe.iloc[idx]['Emotion']], dtype=torch.long)
        sarcasm_label = torch.tensor(self.sarcasm_mapping[self.dataframe.iloc[idx]['Sarcasm']], dtype=torch.float)  # Binary sarcasm
        bully_label = torch.tensor(self.text_label_mapping[self.dataframe.iloc[idx]['Img_Label']], dtype=torch.long)  # Bully detection
        harmful_score_label = torch.tensor(self.harmful_score_mapping[self.dataframe.iloc[idx]['Harmful_Score']], dtype=torch.long)
        target_label = torch.tensor(self.target_mapping[self.dataframe.iloc[idx]['Target']], dtype=torch.long)
        
        return image, inputs['input_ids'].squeeze(), inputs['attention_mask'].squeeze(), sentiment_label, emotion_label, sarcasm_label, bully_label, harmful_score_label, target_label


In [32]:
class Image_text(nn.Module):
    def __init__(self):
        super(Image_text, self).__init__()
        
        # Visual branch (CNN)
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove the final classification layer
        
        # Textual branch (RoBERTa)
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        
        # Shared fully connected layers
        self.fc_shared = nn.Sequential(
            nn.Linear(2048 + 768, 1024),  # Concatenation of visual and textual features
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),  # Deeper layer
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        # Task-specific heads
        self.sentiment_head = nn.Linear(512, 3)  # Sentiment: 3 classes
        self.emotion_head = nn.Linear(512, 10)    # Emotion: 6 classes
        self.sarcasm_head = nn.Linear(512, 1)    # Sarcasm: binary classification
        self.bully_head = nn.Linear(512, 2)      # Cyberbullying: 2 classes
        self.harmful_head = nn.Linear(512, 3)
        self.target_head = nn.Linear(512, 4)
    def forward(self, image, text_input_ids, text_attention_mask):
        # Visual features
        img_features = self.resnet(image)
        
        # Textual features
        text_outputs = self.roberta(input_ids=text_input_ids, attention_mask=text_attention_mask)
        text_features = text_outputs.pooler_output
        
        # Concatenate the visual and textual features
        combined_features = torch.cat((img_features, text_features), dim=1)
        
        # Shared layers
        shared_out = self.fc_shared(combined_features)
        
        # Task-specific outputs
        sentiment_out = self.sentiment_head(shared_out)
        emotion_out = self.emotion_head(shared_out)
        sarcasm_out = torch.sigmoid(self.sarcasm_head(shared_out))
        bully_out = self.bully_head(shared_out)
        harmful_out = self.harmful_head(shared_out)
        target_out = self.target_head(shared_out)
        
        return sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_out, target_out


In [60]:
class Image_text_emotion(nn.Module):
    def __init__(self):
        super(Image_text_emotion, self).__init__()
        
        # Visual branch (CNN)
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove the final classification layer
        
        # Textual branch (RoBERTa)
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        
        # Shared fully connected layers
        self.fc_shared = nn.Sequential(
            nn.Linear(2048 + 768, 1024),  # Concatenation of visual and textual features
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),  # Deeper layer
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        # Task-specific heads
        self.sentiment_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 3)  # Sentiment: 3 classes
        )
        
        self.emotion_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 3)  # Emotion: 3 classes
        )
        
        self.sarcasm_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),  # Sarcasm: binary classification
            nn.Sigmoid()  # Ensures output is in (0, 1)
        )
        # self.bully_head = nn.Linear(512, 2)  # Bully: binary classification
        self.bully_fc = nn.Sequential(
            nn.Linear(3 + 512, 256),  # Input: all task outputs + shared features
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 2)  # 2 classes for bully
        )
        # self.harmful_head = nn.Linear(512, 3)  # Harmful score: 3 classes
        self.harmful_fc = nn.Sequential(
            nn.Linear(3 + 512, 256),  # Input: all task outputs + shared features
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 3)  # 3 classes for harmful
        )
        # Final target head
        self.target_fc = nn.Sequential(
            nn.Linear(3 + 512, 256),  # Input: all task outputs + shared features
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 4)  # 4 classes for Target
        )
        
    def forward(self, image, text_input_ids, text_attention_mask):
        # Visual features
        img_features = self.resnet(image)
        
        # Textual features
        text_outputs = self.roberta(input_ids=text_input_ids, attention_mask=text_attention_mask)
        text_features = text_outputs.pooler_output
        
        # Concatenate the visual and textual features
        combined_features = torch.cat((img_features, text_features), dim=1)
        
        # Shared features
        shared_out = self.fc_shared(combined_features)
        
        # Task-specific predictions
        sentiment_out = self.sentiment_head(shared_out)
        emotion_out = self.emotion_head(shared_out)
        sarcasm_out = torch.sigmoid(self.sarcasm_head(shared_out))  # Binary
        # bully_out = self.bully_head(shared_out)
        # harmful_out = self.harmful_head(shared_out)
        
        # Concatenate all task outputs with shared features for target prediction
        aux_features = torch.cat((
            emotion_out,
            shared_out  # Shared features
        ), dim=1)
        
        # Final target prediction
        bully_out = self.bully_fc(aux_features)
        harmful_out = self.harmful_fc(aux_features)
        target_out = self.target_fc(aux_features)
        
        return sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_out, target_out


In [64]:
class Image_text_sentiment(nn.Module):
    def __init__(self):
        super(Image_text_sentiment, self).__init__()
        
        # Visual branch (CNN)
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove the final classification layer
        
        # Textual branch (RoBERTa)
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        
        # Shared fully connected layers
        self.fc_shared = nn.Sequential(
            nn.Linear(2048 + 768, 1024),  # Concatenation of visual and textual features
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),  # Deeper layer
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        # Task-specific heads
        self.sentiment_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 3)  # Sentiment: 3 classes
        )
        
        self.emotion_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 3)  # Emotion: 3 classes
        )
        
        self.sarcasm_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),  # Sarcasm: binary classification
            nn.Sigmoid()  # Ensures output is in (0, 1)
        )
        
        self.bully_fc = nn.Sequential(
            nn.Linear(3 + 512, 256),  # Input: all task outputs + shared features
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),  # Deeper layer
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 2)  # 2 classes for bully
        )
        
        self.harmful_fc = nn.Sequential(
            nn.Linear(3+ 512, 256),  # Input: all task outputs + shared features
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),  # Deeper layer
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 3)  # 3 classes for harmful
        )
        
        self.target_fc = nn.Sequential(
            nn.Linear(3 + 512, 256),  # Input: all task outputs + shared features
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),  # Deeper layer
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 4)  # 4 classes for Target
        )
        
    def forward(self, image, text_input_ids, text_attention_mask):
        # Visual features
        img_features = self.resnet(image)
        
        # Textual features
        text_outputs = self.roberta(input_ids=text_input_ids, attention_mask=text_attention_mask)
        text_features = text_outputs.pooler_output
        
        # Concatenate the visual and textual features
        combined_features = torch.cat((img_features, text_features), dim=1)
        
        # Shared features
        shared_out = self.fc_shared(combined_features)
        
        # Task-specific predictions
        sentiment_out = self.sentiment_head(shared_out)
        emotion_out = self.emotion_head(shared_out)
        sarcasm_out = self.sarcasm_head(shared_out)
        
        # Concatenate all task outputs with shared features for target prediction
        aux_features = torch.cat((
            sentiment_out,
            shared_out  # Shared features
        ), dim=1)
        
        # Final target prediction
        bully_out = self.bully_fc(aux_features)
        harmful_out = self.harmful_fc(aux_features)
        target_out = self.target_fc(aux_features)
        
        return sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_out, target_out


In [62]:
class Image_text_emotion_sentiment(nn.Module):
    def __init__(self):
        super(Image_text_emotion_sentiment, self).__init__()
        
        # Visual branch (CNN)
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove the final classification layer
        
        # Textual branch (RoBERTa)
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        
        # Shared fully connected layers
        self.fc_shared = nn.Sequential(
            nn.Linear(2048 + 768, 1024),  # Concatenation of visual and textual features
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),  # Deeper layer
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        # Task-specific heads
        self.sentiment_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 3)  # Sentiment: 3 classes
        )
        
        self.emotion_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 3)  # Emotion: 3 classes
        )
        
        self.sarcasm_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),  # Sarcasm: binary classification
            nn.Sigmoid()  # Ensures output is in (0, 1)
        )
        # self.bully_head = nn.Linear(512, 2)  # Bully: binary classification
        self.bully_fc = nn.Sequential(
            nn.Linear(3 + 3 + 512, 256),  # Input: all task outputs + shared features
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 2)  # 2 classes for bully
        )
        # self.harmful_head = nn.Linear(512, 3)  # Harmful score: 3 classes
        self.harmful_fc = nn.Sequential(
            nn.Linear(3 + 3+ 512, 256),  # Input: all task outputs + shared features
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 3)  # 3 classes for harmful
        )
        # Final target head
        self.target_fc = nn.Sequential(
            nn.Linear(3 + 3 +  512, 256),  # Input: all task outputs + shared features
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 4)  # 4 classes for Target
        )
        
    def forward(self, image, text_input_ids, text_attention_mask):
        # Visual features
        img_features = self.resnet(image)
        
        # Textual features
        text_outputs = self.roberta(input_ids=text_input_ids, attention_mask=text_attention_mask)
        text_features = text_outputs.pooler_output
        
        # Concatenate the visual and textual features
        combined_features = torch.cat((img_features, text_features), dim=1)
        
        # Shared features
        shared_out = self.fc_shared(combined_features)
        
        # Task-specific predictions
        sentiment_out = self.sentiment_head(shared_out)
        emotion_out = self.emotion_head(shared_out)
        sarcasm_out = torch.sigmoid(self.sarcasm_head(shared_out))  # Binary
        # bully_out = self.bully_head(shared_out)
        # harmful_out = self.harmful_head(shared_out)
        
        # Concatenate all task outputs with shared features for target prediction
        aux_features = torch.cat((
            emotion_out,
            sentiment_out,
            shared_out  # Shared features
        ), dim=1)
        
        # Final target prediction
        bully_out = self.bully_fc(aux_features)
        harmful_out = self.harmful_fc(aux_features)
        target_out = self.target_fc(aux_features)
        
        return sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_out, target_out


In [36]:
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score, f1_score

In [42]:
dataset = MemeDataset(df, transform)

# Create a DataLoader for batching
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Iterate through the dataloader
for batch in dataloader:
    print(f"Len: {len(batch)}") 
    # Further model training steps...
    break

Len: 9


In [43]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create the dataset and dataloader
dataset = MemeDataset(df, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# dataloader = DataLoader(dataset, batch_size=16, shuffle=True)  # Adjust batch size as needed




# Image and text

In [44]:
# Initialize the model and move it to the GPU
model= Image_text()
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)  # Use DataParallel if multiple GPUs are available
model.to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# Loss functions
loss_fn_sentiment = nn.CrossEntropyLoss().to(device)
loss_fn_emotion = nn.CrossEntropyLoss().to(device)
loss_fn_sarcasm = nn.BCEWithLogitsLoss().to(device)
loss_fn_bully = nn.CrossEntropyLoss().to(device)
loss_fn_harmful_score = nn.CrossEntropyLoss().to(device)
loss_fn_target = nn.CrossEntropyLoss().to(device)

# Training loop
for epoch in range(15):  # Set epochs accordingly
    model.train()
    
    total_loss = 0  # Initialize total loss for the epoch
    
    for images, text_input_ids, text_attention_mask, sentiment_labels, emotion_labels, sarcasm_labels, bully_labels, harmful_score_labels, target_labels in train_dataloader:
        # Move data to the GPU
    # for images, text_input_ids, text_attention_mask, bully_labels in train_dataloader:
  
        images = images.to(device)
        text_input_ids = text_input_ids.to(device)
        text_attention_mask = text_attention_mask.to(device)
        sentiment_labels = sentiment_labels.to(device)
        emotion_labels = emotion_labels.to(device)
        sarcasm_labels = sarcasm_labels.to(device)
        bully_labels = bully_labels.to(device)
        harmful_score_labels = harmful_score_labels.to(device)
        target_labels = target_labels.to(device)
        
        optimizer.zero_grad()  # Clear gradients at the start of each batch
        
        # Forward pass
        sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_score_out, target_out = model(images, text_input_ids, text_attention_mask)
        
        # Compute loss for each task
        loss_sentiment = loss_fn_sentiment(sentiment_out, sentiment_labels)
        loss_emotion = loss_fn_emotion(emotion_out, emotion_labels)
        loss_sarcasm = loss_fn_sarcasm(sarcasm_out.squeeze(), sarcasm_labels.float())  # Squeeze if necessary
        loss_bully = loss_fn_bully(bully_out, bully_labels)
        loss_harmful_score = loss_fn_harmful_score(harmful_score_out, harmful_score_labels)
        loss_target = loss_fn_target(target_out, target_labels)
        
        # Total loss (sum or weigh the losses as needed)
        total_loss_batch = loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target
        
        # Backward pass and optimization
        total_loss_batch.backward()
        optimizer.step()  # Update model parameters
        
        total_loss += total_loss_batch.item()  # Accumulate loss for the epoch

    # Optionally clear cache at the end of each epoch
    torch.cuda.empty_cache()  
    
    # Print the average loss for the epoch
    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}')


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Epoch 0, Average Loss: 3.6535
Epoch 1, Average Loss: 3.3180
Epoch 2, Average Loss: 3.1197
Epoch 3, Average Loss: 2.8727
Epoch 4, Average Loss: 2.5826
Epoch 5, Average Loss: 2.3852
Epoch 6, Average Loss: 2.1845
Epoch 7, Average Loss: 2.0274
Epoch 8, Average Loss: 1.8677
Epoch 9, Average Loss: 1.7558
Epoch 10, Average Loss: 1.6838
Epoch 11, Average Loss: 1.6833
Epoch 12, Average Loss: 1.5604
Epoch 13, Average Loss: 1.5469
Epoch 14, Average Loss: 1.5032


In [50]:
from tqdm import tqdm

In [52]:
def train_model_v1(model, optimizer, losses, epochs=10):
    # Training loop
    for epoch in range(epochs):  # Set epochs accordingly
        model.train()
        
        total_loss = 0  # Initialize total loss for the epoch
        batch_iterator = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f'Epoch [{epoch+1}/{epochs}]')
        for batch_idx, (images, text_input_ids, text_attention_mask, sentiment_labels, emotion_labels, sarcasm_labels, bully_labels, harmful_score_labels, target_labels) in batch_iterator:
            # Move data to the GPU
        # for images, text_input_ids, text_attention_mask, bully_labels in train_dataloader:
      
            images = images.to(device)
            text_input_ids = text_input_ids.to(device)
            text_attention_mask = text_attention_mask.to(device)
            sentiment_labels = sentiment_labels.to(device)
            emotion_labels = emotion_labels.to(device)
            sarcasm_labels = sarcasm_labels.to(device)
            bully_labels = bully_labels.to(device)
            harmful_score_labels = harmful_score_labels.to(device)
            target_labels = target_labels.to(device)
            
            optimizer.zero_grad()  # Clear gradients at the start of each batch
            
            # Forward pass
            sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_score_out, target_out = model(images, text_input_ids, text_attention_mask)
            
            # Compute loss for each task
            loss_sentiment = loss_fn_sentiment(sentiment_out, sentiment_labels)
            loss_emotion = loss_fn_emotion(emotion_out, emotion_labels)
            loss_sarcasm = loss_fn_sarcasm(sarcasm_out.squeeze(), sarcasm_labels.float())  # Squeeze if necessary
            loss_bully = loss_fn_bully(bully_out, bully_labels)
            loss_harmful_score = loss_fn_harmful_score(harmful_score_out, harmful_score_labels)
            loss_target = loss_fn_target(target_out, target_labels)
            
            # Total loss (sum or weigh the losses as needed)
            total_loss_batch = loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target
            
            # Backward pass and optimization
            total_loss_batch.backward()
            optimizer.step()  # Update model parameters
            
            total_loss += total_loss_batch.item()  # Accumulate loss for the epoch
            batch_iterator.set_postfix({'Train Batch Loss': total_loss, 'Training Avg Loss': total_loss_batch.item()/ (batch_idx + 1)})
        # Optionally clear cache at the end of each epoch
        torch.cuda.empty_cache()  
        
        # Print the average loss for the epoch
        avg_loss = total_loss / len(train_dataloader)
        losses.append(avg_loss)
        print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}')

In [53]:
losses = []
train_model_v1(model, optimizer, losses, epochs=10)

Epoch [1/10]: 100%|██████████| 154/154 [01:16<00:00,  2.02it/s, Train Batch Loss=212, Training Avg Loss=0.00745]


Epoch 0, Average Loss: 1.3758


Epoch [2/10]: 100%|██████████| 154/154 [01:15<00:00,  2.04it/s, Train Batch Loss=213, Training Avg Loss=0.0127] 


Epoch 1, Average Loss: 1.3827


Epoch [3/10]: 100%|██████████| 154/154 [01:15<00:00,  2.05it/s, Train Batch Loss=204, Training Avg Loss=0.00701]


...

Epoch 7, Average Loss: 1.1849


Epoch [9/10]: 100%|██████████| 154/154 [01:15<00:00,  2.03it/s, Train Batch Loss=183, Training Avg Loss=0.00737]


Epoch 8, Average Loss: 1.1896


Epoch [10/10]: 100%|██████████| 154/154 [01:15<00:00,  2.04it/s, Train Batch Loss=176, Training Avg Loss=0.00856]

Epoch 9, Average Loss: 1.1438


In [45]:
import numpy as np

In [54]:
# Import required libraries (if not already done)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score

# Validation phase
model.eval()  # Set model to evaluation mode
total_val_loss = 0

# Initialize lists to store true and predicted labels for each task
all_labels_bully = []
all_preds_bully = []

all_labels_sentiment = []
all_preds_sentiment = []

all_labels_emotion = []
all_preds_emotion = []

all_labels_sarcasm = []
all_preds_sarcasm = []

all_labels_harmful_score = []
all_preds_harmful_score = []

all_labels_target = []
all_preds_target = []

with torch.no_grad():  # Disable gradient calculation
    batch_iterator = tqdm(val_dataloader, desc=f"Processing Validation {epoch:02d}")
    for images, text_input_ids, text_attention_mask, sentiment_labels, emotion_labels, sarcasm_labels, bully_labels, harmful_score_labels, target_labels in batch_iterator:
        # Move data to the GPU
        images = images.to(device)
        text_input_ids = text_input_ids.to(device)
        text_attention_mask = text_attention_mask.to(device)
        sentiment_labels = sentiment_labels.to(device)
        emotion_labels = emotion_labels.to(device)
        sarcasm_labels = sarcasm_labels.to(device)
        bully_labels = bully_labels.to(device)
        harmful_score_labels = harmful_score_labels.to(device)
        target_labels = target_labels.to(device)

        # Forward pass
        sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_score_out, target_out = model(images, text_input_ids, text_attention_mask)

        # Compute loss for each task
        loss_sentiment = loss_fn_sentiment(sentiment_out, sentiment_labels)
        loss_emotion = loss_fn_emotion(emotion_out, emotion_labels)
        loss_sarcasm = loss_fn_sarcasm(sarcasm_out.squeeze(), sarcasm_labels.float())
        loss_bully = loss_fn_bully(bully_out, bully_labels)
        loss_harmful_score = loss_fn_harmful_score(harmful_score_out, harmful_score_labels)
        loss_target = loss_fn_target(target_out, target_labels)

        # Total loss
        total_val_loss += (loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target).item()

        # Get predictions for each task
        _, predicted_sentiment = torch.max(sentiment_out, 1)
        _, predicted_emotion = torch.max(emotion_out, 1)
        _, predicted_sarcasm = torch.max(sarcasm_out, 1)
        _, predicted_bully = torch.max(bully_out, 1)
        _, predicted_harmful_score = torch.max(harmful_score_out, 1)  # Assuming multi-class
        _, predicted_target = torch.max(target_out, 1)  # Assuming multi-class

        # Collect true and predicted labels for each task
        all_labels_sentiment.append(sentiment_labels.cpu().numpy())
        all_preds_sentiment.append(predicted_sentiment.cpu().numpy())

        all_labels_emotion.append(emotion_labels.cpu().numpy())
        all_preds_emotion.append(predicted_emotion.cpu().numpy())

        all_labels_sarcasm.append(sarcasm_labels.cpu().numpy())
        all_preds_sarcasm.append(predicted_sarcasm.cpu().numpy())

        all_labels_bully.append(bully_labels.cpu().numpy())
        all_preds_bully.append(predicted_bully.cpu().numpy())

        all_labels_harmful_score.append(harmful_score_labels.cpu().numpy())
        all_preds_harmful_score.append(predicted_harmful_score.cpu().numpy())

        all_labels_target.append(target_labels.cpu().numpy())
        all_preds_target.append(predicted_target.cpu().numpy())

avg_val_loss = total_val_loss / len(val_dataloader)

# Flatten lists for each task
all_labels_bully = np.concatenate(all_labels_bully)
all_preds_bully = np.concatenate(all_preds_bully)

all_labels_sentiment = np.concatenate(all_labels_sentiment)
all_preds_sentiment = np.concatenate(all_preds_sentiment)

all_labels_emotion = np.concatenate(all_labels_emotion)
all_preds_emotion = np.concatenate(all_preds_emotion)

all_labels_sarcasm = np.concatenate(all_labels_sarcasm)
all_preds_sarcasm = np.concatenate(all_preds_sarcasm)

all_labels_harmful_score = np.concatenate(all_labels_harmful_score)
all_preds_harmful_score = np.concatenate(all_preds_harmful_score)

all_labels_target = np.concatenate(all_labels_target)
all_preds_target = np.concatenate(all_preds_target)

# Calculate accuracy and F1 score for each task
accuracy_bully = accuracy_score(all_labels_bully, all_preds_bully)
f1_bully = f1_score(all_labels_bully, all_preds_bully, average='weighted')

accuracy_sentiment = accuracy_score(all_labels_sentiment, all_preds_sentiment)
f1_sentiment = f1_score(all_labels_sentiment, all_preds_sentiment, average='weighted')

accuracy_emotion = accuracy_score(all_labels_emotion, all_preds_emotion)
f1_emotion = f1_score(all_labels_emotion, all_preds_emotion, average='weighted')

accuracy_sarcasm = accuracy_score(all_labels_sarcasm, all_preds_sarcasm)
f1_sarcasm = f1_score(all_labels_sarcasm, all_preds_sarcasm, average='weighted')

accuracy_harmful_score = accuracy_score(all_labels_harmful_score, all_preds_harmful_score)
f1_harmful_score = f1_score(all_labels_harmful_score, all_preds_harmful_score, average='weighted')

accuracy_target = accuracy_score(all_labels_target, all_preds_target)
f1_target = f1_score(all_labels_target, all_preds_target, average='weighted')

print(f'Epoch {epoch}, Validation Loss: {avg_val_loss:.4f},\n'
      f'Bully Accuracy: {accuracy_bully:.4f}, F1 Score: {f1_bully:.4f},\n'
      f'Sentiment Accuracy: {accuracy_sentiment:.4f}, F1 Score: {f1_sentiment:.4f},\n'
      f'Emotion Accuracy: {accuracy_emotion:.4f}, F1 Score: {f1_emotion:.4f},\n'
      f'Sarcasm Accuracy: {accuracy_sarcasm:.4f}, F1 Score: {f1_sarcasm:.4f},\n'
      f'Harmful Score Accuracy: {accuracy_harmful_score:.4f}, F1 Score: {f1_harmful_score:.4f},\n'
      f'Target Accuracy: {accuracy_target:.4f}, F1 Score: {f1_target:.4f}')




Processing Validation 14: 100%|██████████| 39/39 [00:07<00:00,  5.49it/s]

Epoch 14, Validation Loss: 6.4291,
Bully Accuracy: 0.7484, F1 Score: 0.7473,
Sentiment Accuracy: 0.6964, F1 Score: 0.6517,
Emotion Accuracy: 0.6558, F1 Score: 0.5974,
Sarcasm Accuracy: 0.5081, F1 Score: 0.3424,
Harmful Score Accuracy: 0.9805, F1 Score: 0.9805,
Target Accuracy: 0.7419, F1 Score: 0.7230


# Image_text_emotion

In [57]:
# Initialize the model and move it to the GPU
model= Image_text_emotion()
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)  # Use DataParallel if multiple GPUs are available
model.to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# Loss functions
loss_fn_sentiment = nn.CrossEntropyLoss().to(device)
loss_fn_emotion = nn.CrossEntropyLoss().to(device)
loss_fn_sarcasm = nn.BCEWithLogitsLoss().to(device)
loss_fn_bully = nn.CrossEntropyLoss().to(device)
loss_fn_harmful_score = nn.CrossEntropyLoss().to(device)
loss_fn_target = nn.CrossEntropyLoss().to(device)

epochs = 25
# Training loop
for epoch in range(epochs):  # Set epochs accordingly
    model.train()
    
    total_loss = 0  # Initialize total loss for the epoch
    
    batch_iterator = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f'Epoch [{epoch+1}/{epochs}]')
    for batch_idx, (images, text_input_ids, text_attention_mask, sentiment_labels, emotion_labels, sarcasm_labels, bully_labels, harmful_score_labels, target_labels) in batch_iterator:
        # Move data to the GPU
    # for images, text_input_ids, text_attention_mask, bully_labels in train_dataloader:
  
        images = images.to(device)
        text_input_ids = text_input_ids.to(device)
        text_attention_mask = text_attention_mask.to(device)
        sentiment_labels = sentiment_labels.to(device)
        emotion_labels = emotion_labels.to(device)
        sarcasm_labels = sarcasm_labels.to(device)
        bully_labels = bully_labels.to(device)
        harmful_score_labels = harmful_score_labels.to(device)
        target_labels = target_labels.to(device)
        
        optimizer.zero_grad()  # Clear gradients at the start of each batch
        
        # Forward pass
        sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_score_out, target_out = model(images, text_input_ids, text_attention_mask)
        
        # Compute loss for each task
        loss_sentiment = loss_fn_sentiment(sentiment_out, sentiment_labels)
        loss_emotion = loss_fn_emotion(emotion_out, emotion_labels)
        loss_sarcasm = loss_fn_sarcasm(sarcasm_out.squeeze(), sarcasm_labels.float())  # Squeeze if necessary
        loss_bully = loss_fn_bully(bully_out, bully_labels)
        loss_harmful_score = loss_fn_harmful_score(harmful_score_out, harmful_score_labels)
        loss_target = loss_fn_target(target_out, target_labels)
        
        # Total loss (sum or weigh the losses as needed)
        total_loss_batch = loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target
        
        # Backward pass and optimization
        total_loss_batch.backward()
        optimizer.step()  # Update model parameters
        
        total_loss += total_loss_batch.item()  # Accumulate loss for the epoch
        batch_iterator.set_postfix({'Train Batch Loss': total_loss, 'Training Avg Loss': total_loss_batch.item()/ (batch_idx + 1)})
    # Optionally clear cache at the end of each epoch
    torch.cuda.empty_cache()  
    
    # Print the average loss for the epoch
    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}')


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch [1/25]: 100%|██████████| 154/154 [01:16<00:00,  2.02it/s, Train Batch Loss=560, Training Avg Loss=0.0165]


Epoch 0, Average Loss: 3.6347


Epoch [2/25]: 100%|██████████| 154/154 [01:15<00:00,  2.05it/s, Train Batch Loss=518, Training Avg Loss=0.0171]


Epoch 1, Average Loss: 3.3653


Epoch [3/25]: 100%|██████████| 154/154 [01:15<00:00,  2.04it/s, Train Batch Loss=494, Training Avg Loss=0.0177]


...

Epoch 22, Average Loss: 1.5422


Epoch [24/25]: 100%|██████████| 154/154 [01:15<00:00,  2.05it/s, Train Batch Loss=236, Training Avg Loss=0.0115] 


Epoch 23, Average Loss: 1.5344


Epoch [25/25]: 100%|██████████| 154/154 [01:15<00:00,  2.05it/s, Train Batch Loss=227, Training Avg Loss=0.0205] 

Epoch 24, Average Loss: 1.4760


In [58]:
# Import required libraries (if not already done)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score

# Validation phase
model.eval()  # Set model to evaluation mode
total_val_loss = 0

# Initialize lists to store true and predicted labels for each task
all_labels_bully = []
all_preds_bully = []

all_labels_sentiment = []
all_preds_sentiment = []

all_labels_emotion = []
all_preds_emotion = []

all_labels_sarcasm = []
all_preds_sarcasm = []

all_labels_harmful_score = []
all_preds_harmful_score = []

all_labels_target = []
all_preds_target = []

with torch.no_grad():  # Disable gradient calculation
    batch_iterator = tqdm(val_dataloader, desc=f"Processing Validation {epoch:02d}")
    for images, text_input_ids, text_attention_mask, sentiment_labels, emotion_labels, sarcasm_labels, bully_labels, harmful_score_labels, target_labels in batch_iterator:
        # Move data to the GPU
        images = images.to(device)
        text_input_ids = text_input_ids.to(device)
        text_attention_mask = text_attention_mask.to(device)
        sentiment_labels = sentiment_labels.to(device)
        emotion_labels = emotion_labels.to(device)
        sarcasm_labels = sarcasm_labels.to(device)
        bully_labels = bully_labels.to(device)
        harmful_score_labels = harmful_score_labels.to(device)
        target_labels = target_labels.to(device)

        # Forward pass
        sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_score_out, target_out = model(images, text_input_ids, text_attention_mask)

        # Compute loss for each task
        loss_sentiment = loss_fn_sentiment(sentiment_out, sentiment_labels)
        loss_emotion = loss_fn_emotion(emotion_out, emotion_labels)
        loss_sarcasm = loss_fn_sarcasm(sarcasm_out.squeeze(), sarcasm_labels.float())
        loss_bully = loss_fn_bully(bully_out, bully_labels)
        loss_harmful_score = loss_fn_harmful_score(harmful_score_out, harmful_score_labels)
        loss_target = loss_fn_target(target_out, target_labels)

        # Total loss
        total_val_loss += (loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target).item()

        # Get predictions for each task
        _, predicted_sentiment = torch.max(sentiment_out, 1)
        _, predicted_emotion = torch.max(emotion_out, 1)
        _, predicted_sarcasm = torch.max(sarcasm_out, 1)
        _, predicted_bully = torch.max(bully_out, 1)
        _, predicted_harmful_score = torch.max(harmful_score_out, 1)  # Assuming multi-class
        _, predicted_target = torch.max(target_out, 1)  # Assuming multi-class

        # Collect true and predicted labels for each task
        all_labels_sentiment.append(sentiment_labels.cpu().numpy())
        all_preds_sentiment.append(predicted_sentiment.cpu().numpy())

        all_labels_emotion.append(emotion_labels.cpu().numpy())
        all_preds_emotion.append(predicted_emotion.cpu().numpy())

        all_labels_sarcasm.append(sarcasm_labels.cpu().numpy())
        all_preds_sarcasm.append(predicted_sarcasm.cpu().numpy())

        all_labels_bully.append(bully_labels.cpu().numpy())
        all_preds_bully.append(predicted_bully.cpu().numpy())

        all_labels_harmful_score.append(harmful_score_labels.cpu().numpy())
        all_preds_harmful_score.append(predicted_harmful_score.cpu().numpy())

        all_labels_target.append(target_labels.cpu().numpy())
        all_preds_target.append(predicted_target.cpu().numpy())

avg_val_loss = total_val_loss / len(val_dataloader)

# Flatten lists for each task
all_labels_bully = np.concatenate(all_labels_bully)
all_preds_bully = np.concatenate(all_preds_bully)

all_labels_sentiment = np.concatenate(all_labels_sentiment)
all_preds_sentiment = np.concatenate(all_preds_sentiment)

all_labels_emotion = np.concatenate(all_labels_emotion)
all_preds_emotion = np.concatenate(all_preds_emotion)

all_labels_sarcasm = np.concatenate(all_labels_sarcasm)
all_preds_sarcasm = np.concatenate(all_preds_sarcasm)

all_labels_harmful_score = np.concatenate(all_labels_harmful_score)
all_preds_harmful_score = np.concatenate(all_preds_harmful_score)

all_labels_target = np.concatenate(all_labels_target)
all_preds_target = np.concatenate(all_preds_target)

# Calculate accuracy and F1 score for each task
accuracy_bully_em = accuracy_score(all_labels_bully, all_preds_bully)
f1_bully_em = f1_score(all_labels_bully, all_preds_bully, average='weighted')

accuracy_sentiment_em = accuracy_score(all_labels_sentiment, all_preds_sentiment)
f1_sentiment_em = f1_score(all_labels_sentiment, all_preds_sentiment, average='weighted')

accuracy_emotion_em = accuracy_score(all_labels_emotion, all_preds_emotion)
f1_emotion_em = f1_score(all_labels_emotion, all_preds_emotion, average='weighted')

accuracy_sarcasm_em = accuracy_score(all_labels_sarcasm, all_preds_sarcasm)
f1_sarcasm_em = f1_score(all_labels_sarcasm, all_preds_sarcasm, average='weighted')

accuracy_harmful_score_em = accuracy_score(all_labels_harmful_score, all_preds_harmful_score)
f1_harmful_score_em = f1_score(all_labels_harmful_score, all_preds_harmful_score, average='weighted')

accuracy_target_em = accuracy_score(all_labels_target, all_preds_target)
f1_target_em = f1_score(all_labels_target, all_preds_target, average='weighted')

print(f'Epoch {epoch}, Validation Loss: {avg_val_loss:.4f},\n'
      f'Bully Accuracy: {accuracy_bully_em:.4f}, F1 Score: {f1_bully_em:.4f},\n'
      f'Sentiment Accuracy: {accuracy_sentiment_em:.4f}, F1 Score: {f1_sentiment_em:.4f},\n'
      f'Emotion Accuracy: {accuracy_emotion_em:.4f}, F1 Score: {f1_emotion_em:.4f},\n'
      f'Sarcasm Accuracy: {accuracy_sarcasm_em:.4f}, F1 Score: {f1_sarcasm_em:.4f},\n'
      f'Harmful Score Accuracy: {accuracy_harmful_score_em:.4f}, F1 Score: {f1_harmful_score_em:.4f},\n'
      f'Target Accuracy: {accuracy_target_em:.4f}, F1 Score: {f1_target_em:.4f}')




Processing Validation 24: 100%|██████████| 39/39 [00:07<00:00,  5.49it/s]

Epoch 24, Validation Loss: 6.1328,
Bully Accuracy: 0.7500, F1 Score: 0.7633,
Sentiment Accuracy: 0.6688, F1 Score: 0.6589,
Emotion Accuracy: 0.6039, F1 Score: 0.5888,
Sarcasm Accuracy: 0.5081, F1 Score: 0.3424,
Harmful Score Accuracy: 0.9870, F1 Score: 0.9838,
Target Accuracy: 0.7208, F1 Score: 0.7150


# Image_text_sentiment

In [65]:
# Initialize the model and move it to the GPU
model= Image_text_sentiment()
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)  # Use DataParallel if multiple GPUs are available
model.to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# Loss functions
loss_fn_sentiment = nn.CrossEntropyLoss().to(device)
loss_fn_emotion = nn.CrossEntropyLoss().to(device)
loss_fn_sarcasm = nn.BCEWithLogitsLoss().to(device)
loss_fn_bully = nn.CrossEntropyLoss().to(device)
loss_fn_harmful_score = nn.CrossEntropyLoss().to(device)
loss_fn_target = nn.CrossEntropyLoss().to(device)

epochs = 25
# Training loop
for epoch in range(epochs):  # Set epochs accordingly
    model.train()
    
    total_loss = 0  # Initialize total loss for the epoch
    batch_iterator = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f'Epoch [{epoch+1}/{epochs}]')
    for batch_idx, (images, text_input_ids, text_attention_mask, sentiment_labels, emotion_labels, sarcasm_labels, bully_labels, harmful_score_labels, target_labels) in batch_iterator:
        # Move data to the GPU
    # for images, text_input_ids, text_attention_mask, bully_labels in train_dataloader:
  
        images = images.to(device)
        text_input_ids = text_input_ids.to(device)
        text_attention_mask = text_attention_mask.to(device)
        sentiment_labels = sentiment_labels.to(device)
        emotion_labels = emotion_labels.to(device)
        sarcasm_labels = sarcasm_labels.to(device)
        bully_labels = bully_labels.to(device)
        harmful_score_labels = harmful_score_labels.to(device)
        target_labels = target_labels.to(device)
        
        optimizer.zero_grad()  # Clear gradients at the start of each batch
        
        # Forward pass
        sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_score_out, target_out = model(images, text_input_ids, text_attention_mask)
        
        # Compute loss for each task
        loss_sentiment = loss_fn_sentiment(sentiment_out, sentiment_labels)
        loss_emotion = loss_fn_emotion(emotion_out, emotion_labels)
        loss_sarcasm = loss_fn_sarcasm(sarcasm_out.squeeze(), sarcasm_labels.float())  # Squeeze if necessary
        loss_bully = loss_fn_bully(bully_out, bully_labels)
        loss_harmful_score = loss_fn_harmful_score(harmful_score_out, harmful_score_labels)
        loss_target = loss_fn_target(target_out, target_labels)
        
        # Total loss (sum or weigh the losses as needed)
        total_loss_batch = loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target
        
        # Backward pass and optimization
        total_loss_batch.backward()
        optimizer.step()  # Update model parameters
        
        total_loss += total_loss_batch.item()  # Accumulate loss for the epoch
        batch_iterator.set_postfix({'Train Batch Loss': total_loss, 'Training Avg Loss': total_loss_batch.item()/ (batch_idx + 1)})
    # Optionally clear cache at the end of each epoch
    torch.cuda.empty_cache()  
    
    # Print the average loss for the epoch
    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}')


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch [1/25]:   0%|          | 0/154 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-pa

Epoch 0, Average Loss: 3.6867


Epoch [2/25]: 100%|██████████| 154/154 [01:15<00:00,  2.03it/s, Train Batch Loss=520, Training Avg Loss=0.0252]


Epoch 1, Average Loss: 3.3748


Epoch [3/25]: 100%|██████████| 154/154 [01:15<00:00,  2.04it/s, Train Batch Loss=500, Training Avg Loss=0.0292]


...

Epoch 22, Average Loss: 1.6017


Epoch [24/25]: 100%|██████████| 154/154 [01:15<00:00,  2.05it/s, Train Batch Loss=236, Training Avg Loss=0.00911]


Epoch 23, Average Loss: 1.5312


Epoch [25/25]: 100%|██████████| 154/154 [01:15<00:00,  2.05it/s, Train Batch Loss=229, Training Avg Loss=0.0125] 

Epoch 24, Average Loss: 1.4880


In [66]:
# Import required libraries (if not already done)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score

# Validation phase
model.eval()  # Set model to evaluation mode
total_val_loss = 0

# Initialize lists to store true and predicted labels for each task
all_labels_bully = []
all_preds_bully = []

all_labels_sentiment = []
all_preds_sentiment = []

all_labels_emotion = []
all_preds_emotion = []

all_labels_sarcasm = []
all_preds_sarcasm = []

all_labels_harmful_score = []
all_preds_harmful_score = []

all_labels_target = []
all_preds_target = []

with torch.no_grad():  # Disable gradient calculation
    batch_iterator = tqdm(val_dataloader, desc=f"Processing Validation {epoch:02d}")
    for images, text_input_ids, text_attention_mask, sentiment_labels, emotion_labels, sarcasm_labels, bully_labels, harmful_score_labels, target_labels in batch_iterator:
        # Move data to the GPU
        images = images.to(device)
        text_input_ids = text_input_ids.to(device)
        text_attention_mask = text_attention_mask.to(device)
        sentiment_labels = sentiment_labels.to(device)
        emotion_labels = emotion_labels.to(device)
        sarcasm_labels = sarcasm_labels.to(device)
        bully_labels = bully_labels.to(device)
        harmful_score_labels = harmful_score_labels.to(device)
        target_labels = target_labels.to(device)

        # Forward pass
        sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_score_out, target_out = model(images, text_input_ids, text_attention_mask)

        # Compute loss for each task
        loss_sentiment = loss_fn_sentiment(sentiment_out, sentiment_labels)
        loss_emotion = loss_fn_emotion(emotion_out, emotion_labels)
        loss_sarcasm = loss_fn_sarcasm(sarcasm_out.squeeze(), sarcasm_labels.float())
        loss_bully = loss_fn_bully(bully_out, bully_labels)
        loss_harmful_score = loss_fn_harmful_score(harmful_score_out, harmful_score_labels)
        loss_target = loss_fn_target(target_out, target_labels)

        # Total loss
        total_val_loss += (loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target).item()

        # Get predictions for each task
        _, predicted_sentiment = torch.max(sentiment_out, 1)
        _, predicted_emotion = torch.max(emotion_out, 1)
        _, predicted_sarcasm = torch.max(sarcasm_out, 1)
        _, predicted_bully = torch.max(bully_out, 1)
        _, predicted_harmful_score = torch.max(harmful_score_out, 1)  # Assuming multi-class
        _, predicted_target = torch.max(target_out, 1)  # Assuming multi-class

        # Collect true and predicted labels for each task
        all_labels_sentiment.append(sentiment_labels.cpu().numpy())
        all_preds_sentiment.append(predicted_sentiment.cpu().numpy())

        all_labels_emotion.append(emotion_labels.cpu().numpy())
        all_preds_emotion.append(predicted_emotion.cpu().numpy())

        all_labels_sarcasm.append(sarcasm_labels.cpu().numpy())
        all_preds_sarcasm.append(predicted_sarcasm.cpu().numpy())

        all_labels_bully.append(bully_labels.cpu().numpy())
        all_preds_bully.append(predicted_bully.cpu().numpy())

        all_labels_harmful_score.append(harmful_score_labels.cpu().numpy())
        all_preds_harmful_score.append(predicted_harmful_score.cpu().numpy())

        all_labels_target.append(target_labels.cpu().numpy())
        all_preds_target.append(predicted_target.cpu().numpy())

avg_val_loss = total_val_loss / len(val_dataloader)

# Flatten lists for each task
all_labels_bully = np.concatenate(all_labels_bully)
all_preds_bully = np.concatenate(all_preds_bully)

all_labels_sentiment = np.concatenate(all_labels_sentiment)
all_preds_sentiment = np.concatenate(all_preds_sentiment)

all_labels_emotion = np.concatenate(all_labels_emotion)
all_preds_emotion = np.concatenate(all_preds_emotion)

all_labels_sarcasm = np.concatenate(all_labels_sarcasm)
all_preds_sarcasm = np.concatenate(all_preds_sarcasm)

all_labels_harmful_score = np.concatenate(all_labels_harmful_score)
all_preds_harmful_score = np.concatenate(all_preds_harmful_score)

all_labels_target = np.concatenate(all_labels_target)
all_preds_target = np.concatenate(all_preds_target)

# Calculate accuracy and F1 score for each task
accuracy_bully_SA = accuracy_score(all_labels_bully, all_preds_bully)
f1_bully_SA = f1_score(all_labels_bully, all_preds_bully, average='weighted')

accuracy_sentiment_SA = accuracy_score(all_labels_sentiment, all_preds_sentiment)
f1_sentiment_SA = f1_score(all_labels_sentiment, all_preds_sentiment, average='weighted')

accuracy_emotion_SA = accuracy_score(all_labels_emotion, all_preds_emotion)
f1_emotion_SA = f1_score(all_labels_emotion, all_preds_emotion, average='weighted')

accuracy_sarcasm_SA = accuracy_score(all_labels_sarcasm, all_preds_sarcasm)
f1_sarcasm_SA = f1_score(all_labels_sarcasm, all_preds_sarcasm, average='weighted')

accuracy_harmful_score_SA = accuracy_score(all_labels_harmful_score, all_preds_harmful_score)
f1_harmful_score_SA = f1_score(all_labels_harmful_score, all_preds_harmful_score, average='weighted')

accuracy_target_SA = accuracy_score(all_labels_target, all_preds_target)
f1_target_SA = f1_score(all_labels_target, all_preds_target, average='weighted')

print(f'Epoch {epoch}, Validation Loss: {avg_val_loss:.4f},\n'
      f'Bully Accuracy: {accuracy_bully_SA:.4f}, F1 Score: {f1_bully_SA:.4f},\n'
      f'Sentiment Accuracy: {accuracy_sentiment_SA:.4f}, F1 Score: {f1_sentiment_SA:.4f},\n'
      f'Emotion Accuracy: {accuracy_emotion_SA:.4f}, F1 Score: {f1_emotion_SA:.4f},\n'
      f'Sarcasm Accuracy: {accuracy_sarcasm_SA:.4f}, F1 Score: {f1_sarcasm_SA:.4f},\n'
      f'Harmful Score Accuracy: {accuracy_harmful_score_SA:.4f}, F1 Score: {f1_harmful_score_SA:.4f},\n'
      f'Target Accuracy: {accuracy_target_SA:.4f}, F1 Score: {f1_target_SA:.4f}')




Processing Validation 24: 100%|██████████| 39/39 [00:07<00:00,  5.40it/s]

Epoch 24, Validation Loss: 5.5873,
Bully Accuracy: 0.7403, F1 Score: 0.7470,
Sentiment Accuracy: 0.6997, F1 Score: 0.6693,
Emotion Accuracy: 0.6153, F1 Score: 0.6042,
Sarcasm Accuracy: 0.5081, F1 Score: 0.3424,
Harmful Score Accuracy: 0.9838, F1 Score: 0.9822,
Target Accuracy: 0.7500, F1 Score: 0.7199


# Image_text_emotion sentiment

In [67]:
# Initialize the model and move it to the GPU
model= Image_text_emotion_sentiment()
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)  # Use DataParallel if multiple GPUs are available
model.to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# Loss functions
loss_fn_sentiment = nn.CrossEntropyLoss().to(device)
loss_fn_emotion = nn.CrossEntropyLoss().to(device)
loss_fn_sarcasm = nn.BCEWithLogitsLoss().to(device)
loss_fn_bully = nn.CrossEntropyLoss().to(device)
loss_fn_harmful_score = nn.CrossEntropyLoss().to(device)
loss_fn_target = nn.CrossEntropyLoss().to(device)

epochs = 25
# Training loop
for epoch in range(epochs):  # Set epochs accordingly
    model.train()
    
    total_loss = 0  # Initialize total loss for the epoch
    batch_iterator = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f'Epoch [{epoch+1}/{epochs}]')
    for batch_idx, (images, text_input_ids, text_attention_mask, sentiment_labels, emotion_labels, sarcasm_labels, bully_labels, harmful_score_labels, target_labels) in batch_iterator:
        # Move data to the GPU
    # for images, text_input_ids, text_attention_mask, bully_labels in train_dataloader:
  
        images = images.to(device)
        text_input_ids = text_input_ids.to(device)
        text_attention_mask = text_attention_mask.to(device)
        sentiment_labels = sentiment_labels.to(device)
        emotion_labels = emotion_labels.to(device)
        sarcasm_labels = sarcasm_labels.to(device)
        bully_labels = bully_labels.to(device)
        harmful_score_labels = harmful_score_labels.to(device)
        target_labels = target_labels.to(device)
        
        optimizer.zero_grad()  # Clear gradients at the start of each batch
        
        # Forward pass
        sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_score_out, target_out = model(images, text_input_ids, text_attention_mask)
        
        # Compute loss for each task
        loss_sentiment = loss_fn_sentiment(sentiment_out, sentiment_labels)
        loss_emotion = loss_fn_emotion(emotion_out, emotion_labels)
        loss_sarcasm = loss_fn_sarcasm(sarcasm_out.squeeze(), sarcasm_labels.float())  # Squeeze if necessary
        loss_bully = loss_fn_bully(bully_out, bully_labels)
        loss_harmful_score = loss_fn_harmful_score(harmful_score_out, harmful_score_labels)
        loss_target = loss_fn_target(target_out, target_labels)
        
        # Total loss (sum or weigh the losses as needed)
        total_loss_batch = loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target
        
        # Backward pass and optimization
        total_loss_batch.backward()
        optimizer.step()  # Update model parameters
        
        total_loss += total_loss_batch.item()  # Accumulate loss for the epoch
        batch_iterator.set_postfix({'Train Batch Loss': total_loss, 'Training Avg Loss': total_loss_batch.item()/ (batch_idx + 1)})
    # Optionally clear cache at the end of each epoch
    torch.cuda.empty_cache()  
    
    # Print the average loss for the epoch
    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}')


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch [1/25]:   0%|          | 0/154 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-pa

Epoch 0, Average Loss: 3.6295


Epoch [2/25]: 100%|██████████| 154/154 [01:15<00:00,  2.05it/s, Train Batch Loss=519, Training Avg Loss=0.0207]


Epoch 1, Average Loss: 3.3689


Epoch [3/25]: 100%|██████████| 154/154 [01:15<00:00,  2.05it/s, Train Batch Loss=494, Training Avg Loss=0.0244]


...

Epoch 22, Average Loss: 1.5660


Epoch [24/25]: 100%|██████████| 154/154 [01:15<00:00,  2.05it/s, Train Batch Loss=238, Training Avg Loss=0.00976]


Epoch 23, Average Loss: 1.5430


Epoch [25/25]: 100%|██████████| 154/154 [01:15<00:00,  2.04it/s, Train Batch Loss=224, Training Avg Loss=0.00877]

Epoch 24, Average Loss: 1.4535


In [68]:
# Import required libraries (if not already done)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score

# Validation phase
model.eval()  # Set model to evaluation mode
total_val_loss = 0

# Initialize lists to store true and predicted labels for each task
all_labels_bully = []
all_preds_bully = []

all_labels_sentiment = []
all_preds_sentiment = []

all_labels_emotion = []
all_preds_emotion = []

all_labels_sarcasm = []
all_preds_sarcasm = []

all_labels_harmful_score = []
all_preds_harmful_score = []

all_labels_target = []
all_preds_target = []

with torch.no_grad():  # Disable gradient calculation
    batch_iterator = tqdm(val_dataloader, desc=f"Processing Validation {epoch:02d}")
    for images, text_input_ids, text_attention_mask, sentiment_labels, emotion_labels, sarcasm_labels, bully_labels, harmful_score_labels, target_labels in batch_iterator:
        # Move data to the GPU
        images = images.to(device)
        text_input_ids = text_input_ids.to(device)
        text_attention_mask = text_attention_mask.to(device)
        sentiment_labels = sentiment_labels.to(device)
        emotion_labels = emotion_labels.to(device)
        sarcasm_labels = sarcasm_labels.to(device)
        bully_labels = bully_labels.to(device)
        harmful_score_labels = harmful_score_labels.to(device)
        target_labels = target_labels.to(device)

        # Forward pass
        sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_score_out, target_out = model(images, text_input_ids, text_attention_mask)

        # Compute loss for each task
        loss_sentiment = loss_fn_sentiment(sentiment_out, sentiment_labels)
        loss_emotion = loss_fn_emotion(emotion_out, emotion_labels)
        loss_sarcasm = loss_fn_sarcasm(sarcasm_out.squeeze(), sarcasm_labels.float())
        loss_bully = loss_fn_bully(bully_out, bully_labels)
        loss_harmful_score = loss_fn_harmful_score(harmful_score_out, harmful_score_labels)
        loss_target = loss_fn_target(target_out, target_labels)

        # Total loss
        total_val_loss += (loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target).item()

        # Get predictions for each task
        _, predicted_sentiment = torch.max(sentiment_out, 1)
        _, predicted_emotion = torch.max(emotion_out, 1)
        _, predicted_sarcasm = torch.max(sarcasm_out, 1)
        _, predicted_bully = torch.max(bully_out, 1)
        _, predicted_harmful_score = torch.max(harmful_score_out, 1)  # Assuming multi-class
        _, predicted_target = torch.max(target_out, 1)  # Assuming multi-class

        # Collect true and predicted labels for each task
        all_labels_sentiment.append(sentiment_labels.cpu().numpy())
        all_preds_sentiment.append(predicted_sentiment.cpu().numpy())

        all_labels_emotion.append(emotion_labels.cpu().numpy())
        all_preds_emotion.append(predicted_emotion.cpu().numpy())

        all_labels_sarcasm.append(sarcasm_labels.cpu().numpy())
        all_preds_sarcasm.append(predicted_sarcasm.cpu().numpy())

        all_labels_bully.append(bully_labels.cpu().numpy())
        all_preds_bully.append(predicted_bully.cpu().numpy())

        all_labels_harmful_score.append(harmful_score_labels.cpu().numpy())
        all_preds_harmful_score.append(predicted_harmful_score.cpu().numpy())

        all_labels_target.append(target_labels.cpu().numpy())
        all_preds_target.append(predicted_target.cpu().numpy())

avg_val_loss = total_val_loss / len(val_dataloader)

# Flatten lists for each task
all_labels_bully = np.concatenate(all_labels_bully)
all_preds_bully = np.concatenate(all_preds_bully)

all_labels_sentiment = np.concatenate(all_labels_sentiment)
all_preds_sentiment = np.concatenate(all_preds_sentiment)

all_labels_emotion = np.concatenate(all_labels_emotion)
all_preds_emotion = np.concatenate(all_preds_emotion)

all_labels_sarcasm = np.concatenate(all_labels_sarcasm)
all_preds_sarcasm = np.concatenate(all_preds_sarcasm)

all_labels_harmful_score = np.concatenate(all_labels_harmful_score)
all_preds_harmful_score = np.concatenate(all_preds_harmful_score)

all_labels_target = np.concatenate(all_labels_target)
all_preds_target = np.concatenate(all_preds_target)

# Calculate accuracy and F1 score for each task
accuracy_bully_SA_EM = accuracy_score(all_labels_bully, all_preds_bully)
f1_bully_SA_EM = f1_score(all_labels_bully, all_preds_bully, average='weighted')

accuracy_sentiment_SA_EM = accuracy_score(all_labels_sentiment, all_preds_sentiment)
f1_sentiment_SA_EM = f1_score(all_labels_sentiment, all_preds_sentiment, average='weighted')

accuracy_emotion_SA_EM = accuracy_score(all_labels_emotion, all_preds_emotion)
f1_emotion_SA_EM = f1_score(all_labels_emotion, all_preds_emotion, average='weighted')

accuracy_sarcasm_SA_EM = accuracy_score(all_labels_sarcasm, all_preds_sarcasm)
f1_sarcasm_SA_EM = f1_score(all_labels_sarcasm, all_preds_sarcasm, average='weighted')

accuracy_harmful_score_SA_EM = accuracy_score(all_labels_harmful_score, all_preds_harmful_score)
f1_harmful_score_SA_EM = f1_score(all_labels_harmful_score, all_preds_harmful_score, average='weighted')

accuracy_target_SA_EM = accuracy_score(all_labels_target, all_preds_target)
f1_target_SA_EM = f1_score(all_labels_target, all_preds_target, average='weighted')

print(f'Epoch {epoch}, Validation Loss: {avg_val_loss:.4f},\n'
      f'Bully Accuracy: {accuracy_bully_SA_EM:.4f}, F1 Score: {f1_bully_SA_EM:.4f},\n'
      f'Sentiment Accuracy: {accuracy_sentiment_SA_EM:.4f}, F1 Score: {f1_sentiment_SA_EM:.4f},\n'
      f'Emotion Accuracy: {accuracy_emotion_SA_EM:.4f}, F1 Score: {f1_emotion_SA_EM:.4f},\n'
      f'Sarcasm Accuracy: {accuracy_sarcasm_SA_EM:.4f}, F1 Score: {f1_sarcasm_SA_EM:.4f},\n'
      f'Harmful Score Accuracy: {accuracy_harmful_score_SA_EM:.4f}, F1 Score: {f1_harmful_score_SA_EM:.4f},\n'
      f'Target Accuracy: {accuracy_target_SA_EM:.4f}, F1 Score: {f1_target_SA_EM:.4f}')




Processing Validation 24: 100%|██████████| 39/39 [00:07<00:00,  5.53it/s]

Epoch 24, Validation Loss: 6.7434,
Bully Accuracy: 0.7727, F1 Score: 0.7591,
Sentiment Accuracy: 0.5438, F1 Score: 0.5711,
Emotion Accuracy: 0.5649, F1 Score: 0.5656,
Sarcasm Accuracy: 0.5081, F1 Score: 0.3424,
Harmful Score Accuracy: 0.9854, F1 Score: 0.9830,
Target Accuracy: 0.7062, F1 Score: 0.7003
